# NLQ with LangGraph ReAct Agents

## Configuration

### Run GraphDB

You need a running GraphDB instance. This tutorial shows how to run the database locally using the GraphDB Docker image. It provides a docker compose set-up, which populates GraphDB with the Star Wars dataset.

- Install [Docker](https://docs.docker.com/get-docker/). This tutorial is created using Docker version 28.0.1 which bundles Docker Compose. For earlier Docker versions you may need to install Docker Compose separately.
- Start GraphDB with the following script executed from the `docker` folder

```
docker build --tag graphdb .
docker compose up -d graphdb
```

You need to wait a couple of seconds for the database to start on http://localhost:7200/. The Star Wars dataset starwars-data.ttl is automatically loaded into the `starwars` repository. The local SPARQL endpoint http://localhost:7200/repositories/starwars can be used to run queries against. You can also open the GraphDB Workbench from your favourite web browser http://localhost:7200/sparql where you can make queries interactively.

### GraphDB

In [ ]:
from base64 import b64encode

from ttyg.graphdb import GraphDB

graphdb_base_url = "http://localhost:7200"
graphdb_repository_id = "starwars"

# If GraphDB is not secured
graph = GraphDB(
    base_url=graphdb_base_url,
    repository_id=graphdb_repository_id,
)

# If GraphDB is secured, you can use the auth_header parameter to pass the value of the "Authorization" header.
# The example below uses a basic authentication.
# username, password = "admin", "root"
# graph = GraphDB(
#     base_url=graphdb_base_url,
#     repository_id=graphdb_repository_id,
#     auth_header="Basic " + b64encode(f"{username}:{password}".encode("ascii")).decode(),
# )

### LLM

#### ChatGPT

In [ ]:
from langchain_openai import ChatOpenAI

from ttyg.utils import set_env

set_env("OPENAI_API_KEY")
gpt_model = ChatOpenAI(
    model="gpt-4o-2024-05-13",
    temperature=0
)

#### Databricks

In [ ]:
import os

from databricks_langchain import ChatDatabricks

from ttyg.utils import set_env

# See [Authentication Documentation](https://docs.databricks.com/en/dev-tools/auth/index.html#databricks-personal-access-tokens) for how to get an access token
set_env("DATABRICKS_TOKEN")
os.environ["DATABRICKS_HOST"] = "https://{workspace-id}.cloud.databricks.com"

llama_model = ChatDatabricks(
    model="databricks-meta-llama-3-1-70b-instruct",
    temperature=0,
)

#### Anthropic

In [ ]:
from langchain_anthropic import ChatAnthropic

from ttyg.utils import set_env

set_env("ANTHROPIC_API_KEY")
claude_model = ChatAnthropic(
    model_name="claude-3-haiku-20240307",
    temperature=0
)

## Define the tools

In [ ]:
from pathlib import Path

from ttyg.tools import (
    FTSTool,
    IRIDiscoveryTool,
    NowTool,
    RetrievalQueryTool,
    SimilaritySearchQueryTool,
    SparqlQueryTool,
)

ontology_schema_file_path = Path("..") / "docker" / "SWAPI-ontology.ttl"
sparql_query_tool = SparqlQueryTool(
    graph=graph,
    ontology_schema_file_path=ontology_schema_file_path,
)

# The full-text search (FTS) must be enabled for the repository in order to use this tool.
# For details how to enable it check the documentation https://graphdb.ontotext.com/documentation/10.8/full-text-search.html#simple-full-text-search-index .
# It's also recommended to compute the RDF rank for the repository.
# For details how to compute it refer to the documentation https://graphdb.ontotext.com/documentation/10.8/ranking-results.html .
fts_tool = FTSTool(
    graph=graph,
)

# The full-text search (FTS) must be enabled for the repository in order to use this tool.
# For details how to enable it check the documentation https://graphdb.ontotext.com/documentation/10.8/full-text-search.html#simple-full-text-search-index .
# It's also recommended to compute the RDF rank for the repository.
# For details how to compute it refer to the documentation https://graphdb.ontotext.com/documentation/10.8/ranking-results.html .
iri_discovery_tool = IRIDiscoveryTool(
    graph=graph,
)

# ChatGPT Retrieval Plugin Connector must exist in order to use this tool.
# In order to set up the ChatGPT Retrieval Connector Tool with an open source LLM, contact Graphwise, doing business as Ontotext, for additional help.
# retrieval_connector_name = "retrievalConnector"
# retrieval_query_tool = RetrievalQueryTool(
#     graph=graph,
#     connector_name=retrieval_connector_name,
# )

# Similarity Index must exist in order to use this tool.
similarity_index_name = "similarityIndex"
similarity_score_threshold = 0.9
similarity_query_tool = SimilaritySearchQueryTool(
    graph=graph,
    index_name=similarity_index_name,
    similarity_score_threshold=similarity_score_threshold,
)

now_tool = NowTool()

## Create the ReAct agent

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

instructions = f"""You are a natural language querying assistant, and you answer the users' questions.
If you need to write a SPARQL query, use only the classes and properties provided in the schema and don't invent or guess any.
Include all the prefixes from the ontology schema in the SPARQL queries.
The ontology schema in turtle format to use in SPARQL queries is:
```turtle
{sparql_query_tool.schema_graph.serialize(format='turtle')}
```
"""

agent_executor = create_react_agent(
    model=gpt_model,
    tools=[
        sparql_query_tool,
        fts_tool,
        iri_discovery_tool,
        # retrieval_query_tool,
        similarity_query_tool,
        now_tool,
    ],
    state_modifier=instructions,
    checkpointer=MemorySaver(),
    # debug=True,
)

## Conversation

In [ ]:
def print_stream(agent, inputs, config):
    for s in agent.stream(inputs, config, stream_mode="values"):
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


conf = {"configurable": {"thread_id": "thread-1"}}
messages = {"messages": [("user", "How many Star Wars movies are there")]}
print_stream(agent_executor, messages, conf)

In [ ]:
messages = {"messages": [("user", "How many awards each of them received")]}
print_stream(agent_executor, messages, conf)

## Create ReAct agent with user confirmation step

In [ ]:
agent_executor = create_react_agent(
    model=gpt_model,
    tools=[
        sparql_query_tool,
        fts_tool,
        iri_discovery_tool,
        # retrieval_query_tool,
        similarity_query_tool,
        now_tool,
    ],
    interrupt_before=["tools"],
    state_modifier=instructions,
    checkpointer=MemorySaver(),
    # debug=True,
)

### Conversation

In [ ]:
conf = {"configurable": {"thread_id": "thread-2"}}
messages = {"messages": [("user", "How many movie characters are in the dataset")]}
print_stream(agent_executor, messages, conf)
snapshot = agent_executor.get_state(conf)
print("Next step: ", snapshot.next)

### Ask for user confirmation

In [ ]:
proceed = input("Do you want to proceed? (yes/no): ")

### Optionally proceed

In [ ]:
if proceed == "yes":
    print_stream(agent_executor, None, conf)

When you're finished playing with NLQ with GraphDB, you can shut down the Docker environment by running 
```
docker compose down -v --remove-orphans
```
from the `docker` directory.